In [2]:
import classiq
from classiq.applications.finance import log_normal_model_input, model_input

num_qubits = 3  #number of qubits will be decided by the number of possible price paths, here we consider 8 possible price paths
mu = 0.7
sigma = 0.13  

# we consider a lognormal distribution of the spot prices with mean and standard deviation defined above
log_normal_model = log_normal_model_input.LogNormalModelInput(
    num_qubits=num_qubits, mu=mu, sigma=sigma
)

distribution_range = log_normal_model.distribution_range #gives the upper and lower bound for the spot prices
print(distribution_range)

(1.2354548886696226, 2.8262271398622736)


The payoff function to describe for example a European call option will be:

$f(S)=\
\Bigg\{\begin{array}{lr}
    0, & \text{when } K\geq S\\
    S - K, & \text{when } K < S\end{array}
$

In [3]:
from classiq.applications.finance import function_input

threshold = 1.9 #this is the strike price taken for the option contract

condition = function_input.FunctionCondition(threshold=threshold, larger=True)
finance_function = function_input.FinanceFunctionInput(
    f="european call option",
    condition=condition,
)
#Here we consider a eurpean call option with a payoff function defined above
print(finance_function)

f=<FinanceFunctionType.EUROPEAN_CALL_OPTION: 3> variable='x' condition=FunctionCondition(threshold=1.9, larger=True) polynomial_degree=None use_chebyshev_polynomial_approximation=False tail_probability=None


In [4]:
from classiq import construct_finance_model

qmod = construct_finance_model(
    finance_model_input=log_normal_model,
    finance_function_input=finance_function,
    phase_port_size=3,
)
#### Loading the model

#Here we are creating defining the logic flow of the quantum algorithm. 
#In this case, the logic flow includes distribution loading and then the payoff function, which is then plugged into the amplitude estimation algorithm (using phase estimation). 
# phase_port_size defines the number of qubits used by the QPE algorithm which will set the accuracy of the calculation.

In [11]:
from classiq import Constraints
from classiq.synthesis import set_constraints

qmod = set_constraints(qmod, constraints=Constraints(max_width=20))
#this sets the number of qubits constraint;
# print(qmod)

In [9]:
# creating a .qmod file and writing the qmod content previously defined on this file.
# We can copy the contents of this file and paste it on the classiq synthesis page and then execute our model there or use the synthesize class of classiq sdk (shown below)
with open("option_pricing.qmod", "w") as f:
    f.write(qmod)
    
print(qmod)

{
  "functions": [
    {
      "name": "qmci",
      "param_decls": {
        "num_phase_qubits": {
          "kind": "int"
        },
        "num_unitary_qubits": {
          "kind": "int"
        }
      },
      "port_declarations": {
        "phase_port": {
          "name": "phase_port",
          "size": {
            "expr": "num_phase_qubits"
          },
          "direction": "output"
        },
        "unitary_port": {
          "name": "unitary_port",
          "size": {
            "expr": "num_unitary_qubits"
          },
          "direction": "output"
        }
      },
      "operand_declarations": {
        "sp_op": {
          "name": "sp_op",
          "param_decls": {
            "num_unitary_qubits": {
              "kind": "int"
            }
          },
          "port_declarations": {
            "reg": {
              "name": "reg",
              "size": {
                "expr": "num_unitary_qubits-1"
              },
              "direction": "inout"
   

In [12]:
from classiq import synthesize

qprog = synthesize(qmod)

In [13]:
from classiq import GeneratedCircuit, show

show(qprog)

Opening: https://platform.classiq.io/circuit/143f2f0a-5d66-43ab-a947-78f64654623b?version=0.31.0


In [14]:
from classiq import GeneratedCircuit
circuit = GeneratedCircuit.from_qprog(qprog)
print(circuit.transpiled_circuit.depth)

3102


In [15]:
from classiq import execute

results = execute(qprog).result()
# print(results)

In [16]:
print(results[1].name, ":", results[1].value)
#this prints out the result estimation of the options price

estimation : 0.37874859780933384
